In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


In [2]:

from transformers import AutoModelForCausalLM, AutoTokenizer

from peft import AutoPeftModelForCausalLM, PeftModel, PeftConfig

In [3]:

model_id = "Langboat/bloom-1b4-zh"  # 原始模型model id
peft_model_id = "/root/autodl-tmp/prompt_tuning_bloom_1b4/checkpoint-1000/"  # 经过微调后模型的加载和使用


In [4]:

config = PeftConfig.from_pretrained("/root/autodl-tmp/prompt_tuning_bloom_1b4/save_pretrained")
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, cache_dir="/root/autodl-tmp/bloom-1b4-zh")

# 默认是推理模式，即is_trainable=False
prompt_tuning_model = PeftModel.from_pretrained(model=model,model_id=peft_model_id,is_trainable=False)




In [5]:

prompt_tuning_model



PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [7]:

prompt_tuning_model = prompt_tuning_model.cuda()
prompt_tuning_model.device



device(type='cuda', index=0)

In [9]:

from transformers import AutoTokenizer

cache_dir = '/root/autodl-tmp/'

tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh", cache_dir=os.path.join(cache_dir, "bloom-1b4-zh"))


ipt = tokenizer("Human: {}\n{}".format("考试有哪些好的技巧", "").strip() + "\n\nAssistant: ", return_tensors='pt').to(prompt_tuning_model.device)


In [11]:

prompt_tuning_model.generate(**ipt, max_length=256, do_sample=True)



tensor([[26283,    29,   210, 12913, 26620,  4775, 16012,   189,   189,  4340,
         17245,    29,   210, 18244, 17799,   584,  7189, 13398,  4775,  4118,
           355, 31212,  2775, 26656,   373,  8583,  7160, 21861,   355,  4573,
          7189, 20677,   420, 11956,  2867, 16610,   554, 10849,   642, 10849,
          1058,  6238,   865, 13165, 27431,   642, 10849, 16012,   355,  1058,
          5706,  4851, 10849,  4155,   420, 10849,  4155,  5574,  6238,  1689,
         24073, 27431,   355,   587, 17799, 10279,  5574,  4573, 27431,   355,
         18516,  5706,  1689, 10849,   373, 12230,   420,  1689, 16645, 17799,
         32299,   355,  5706,  4851, 32299,  8620,   355,  6238,  1689, 14383,
           877,  8715,  1058, 14335,   642,  8971,   355,  3985,  7064, 16430,
          7189,   655, 11293,   620, 16976, 26428,   672, 25078,  6349,  1700,
         17799, 10849,   355, 16645, 17799, 22585,   355, 22585,  5500, 10849,
         36159,  6238,   355,  5574,  6238,  1689, 1

In [14]:
print(tokenizer.decode(prompt_tuning_model.generate(**ipt, max_length=256, do_sample=True)[0], 
                 skip_special_tokens=True))

Human: 考试有哪些好的技巧

Assistant: 阅读理解作为外语考试的阅读部分，通常考察考生的语言知识、语言运用能力和语言综合运用能力等多个方面。阅读理解一般有以下几个方面：
1、首先要懂得阅读的材料。在阅读材料时应该认真审读，做到“通读一遍”，以获得读懂材料的基本信息，“精读一段”，明确文段、段间或段内的主要内容和难点，然后再进行细读、慢读、浏览，以获得理解内容的关键信息。
2、在阅读材料的过程中，对每一句话、每一半句、每一段，要重点地理解，不能单纯地根据句型、语法规则理解，必须重视句的语境、语体的整体性和内容的准确性，还要根据文章的结构、文章的中心思想、作者对文章作者的态度以及对文章内容的认识等方面的信息进行理解。
3、阅读的材料应适合自己的阅读水平和阅读习惯，不能“机械”阅读。要按照自己所学的语种和教学大纲的要求及本学段的要求阅读所选的材料，不要看一些陌生的书或是读一些没有营养的文章，以免误判和理解错误。
4、阅读的材料应具有


In [4]:


# 通过外挂的方式进行加载


origin_model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", cache_dir="/root/autodl-tmp/bloom-1b4-zh")


pp_model = origin_model.load_adapter("/root/autodl-tmp/prompt_tuning_bloom_1b4/save_pretrained")


pp_model


ValueError: `create_and_replace` does not support prompt learning and adaption prompt yet.